In [0]:
# Common Opponent Lister - for any team (myhockeyrankings team id needed), go through all their opponents
# looking for common opponents with Xtreme. Print out a list of common opponents and results for both Xtreme
# and upcoming opponent. 


In [1]:
# install pre-reqs
!pip install pycurl
!pip install beautifulsoup4

# imports 
import pycurl
import pandas as pd 
from bs4 import BeautifulSoup
from io import BytesIO
from google.colab import files


     |████████████████████████████████| 215kB 4.7MB/s 
  Created wheel for pycurl: filename=pycurl-7.43.0.3-cp36-cp36m-linux_x86_64.whl size=282637 sha256=168a9ab9c5b86640ab83f088f06dc4c2f623f1782de22caf65087063dacd9186
  Stored in directory: /root/.cache/pip/wheels/89/40/f5/7b4f2285aca871b5173887a6c69127210d92806c0d3a977e51
Successfully built pycurl


In [0]:
# function to build dataframe with all games
def get_games(id):

  # grab the page for team 
  buffer = BytesIO()
  c = pycurl.Curl()
  c.setopt(c.URL, "https://myhockeyrankings.com/team_info.php?y=2019&t=" + str(id))
  c.setopt(c.WRITEDATA, buffer)
  c.perform()
  c.close()

  # decode into a string we can work with 
  body = buffer.getvalue()
  html = body.decode('iso-8859-1')

  # pass that into beautiful soup 
  soup = BeautifulSoup(html)
  # extract the table will all the results 
  table = soup.find(id="team-scores")

  # loop through each row and add to dataframe
  output_rows = []
  for table_row in table.findAll('tr'):
      columns = table_row.findAll('td')
      output_row = []
      for column in columns:
          output_row.append(column.text)
      output_rows.append(output_row)
      #print(output_row)

  # also, grab the name of the team 
  team_name = soup.find(class_="sitestyle6").text
  #team_name = element.get_text()

  return output_rows, team_name


In [0]:
# get the xtreme game list 
xtreme_id = "2658"
rows, team_name = get_games(xtreme_id)
df_xtreme = pd.DataFrame(rows, columns = ['Date', 'Time', 'Opponent','Location','Score','NA'])


In [0]:
# get the opponent
#opponent_id = "1356" # seacoast spartans elite
#opponent_id = "1306" # east coast wizards elite
#opponent_id = "6332" # woodbridge wolverines
opponent_id = "2674" # bay state breakers
rows, team_name = get_games(opponent_id)
df_opponent = pd.DataFrame(rows, columns = ['Date', 'Time', 'Opponent','Location','Score','NA'])


In [12]:
# shorten down to unique opponents
opp_list  = df_xtreme.Opponent.unique()

# df_common  = df_opponent.Opponent.isin(opp_list)
df_common = df_opponent[df_opponent.Opponent.isin(opp_list)]

# shut off warning
pd.options.mode.chained_assignment = None

# clean up the dataframe by dropping some colums
del df_common["Time"] # don't care 
del df_common["Location"] # don't care 
del df_common["NA"] # don't care
df_common.drop(df_common.head(1).index, inplace=True) # first row always says none for some reason

# remove any spaces in the team name
team_name.replace(" ", "")

# save as csv
df_common.to_csv("/content/" + team_name + "_common_opponents.csv", index=False)

# and display it
df_common

,Date,Opponent,Score
